# TELL Runs for IM3's Experiment Group B 

This notebook executes the initial set of runs of the TELL model for IM3's Experiment Group B.

In [ ]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell

import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

from tell.package_data import get_ba_abbreviations


## Set the Directory Structure

In [ ]:
# Identify the top-level directory and the subdirectory where the data will be stored:
current_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs'
tell_data_dir = os.path.join(current_dir, r'tell_data')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)


## Run the MLP Models

In [ ]:
# Generate a list of BA abbreviations to process:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()

scenario_to_process = 'historic'

# Run the MLP prediction step for the list of BAs using parallel processing streams:
for year_to_process in range(1980,2020,1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                             year = year_to_process,
                             data_dir = os.path.join(tell_data_dir, r'wrf_to_tell_data', scenario_to_process),
                             datetime_field_name = 'Time_UTC',
                             save_prediction = True,
                             prediction_output_directory = os.path.join(tell_data_dir, r'outputs', r'mlp_output', scenario_to_process),
                             n_jobs = -1)


## Run the Forward Execution

In [ ]:
# Run the TELL model forward in time:
for year in range(2020,2099,5):
    summary_df, ba_time_series_df, state_time_series_df = tell.execute_forward(year_to_process = str(year),
                                                                               gcam_target_year = str(year), 
                                                                               scenario_to_process = 'rcp85hotter_ssp5',
                                                                               data_output_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output',
                                                                               gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/gcamusa_data',
                                                                               map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/ba_service_territory_data',
                                                                               mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/mlp_output',
                                                                               pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/population_data',
                                                                               save_county_data = False)
